In [39]:
import csv
import numpy
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
get_ipython().magic('matplotlib inline')


In [40]:
teams_file = "Data//Teams.csv"

In [41]:
path = "Data//Retrosheet//GL"
startyr = 2000
endyr = 2016


In [42]:
# Indicies
# game logs
lscorex, wscorex, parkx, attx  =  8, 7, 16, 14

# teams_file
yrx, teamidx, gx, home_gx, wx, lx = 1, 0, 6, 7, 8, 9
team_namex, avg_attx = 0, 42


In [43]:
def readcsv(fname):
    f = open(fname, 'r')
    reader = csv.reader(f)
    lines = list(reader)
    f.close()
    return lines

In [44]:
class Team:
    def __init__(self, team, teamid, yr):
        self.team = team
        self.teamid = teamid
        self.year = yr
        self.avg_att = 0
        self.att_std = 0
        self.games = []
    def addGame(self, line):
        self.games.append(line)
    def calcStandardDeviation(self):
        game_att = []
        for g in range(0, len(self.games)):
            game = self.games[g]
            att = int(game[attx])
            game_att.append(att)
        std = numpy.std(game_att)
        self.att_std = std
    def setAvgAtt(self):
        num_games = len(self.games)
        total_att = 0
        for i in range(0, num_games):
            total_att += int(self.games[i][attx])
        self.avg_att = total_att / num_games
    def getAttDeviation(self, game):
        deviation = int(game[attx]) - self.avg_att
        return deviation
    def win(self, game):
        if game[hscorex] > game[vscorex]:
            return True
        else:
            return False

In [59]:
def build_teamid_ref(lines):
    d = {}
    sortedLines = sorted(lines, key=lambda y: y[yrx], reverse=True)
    for line in sortedLines:
        teamid = line[teamidx]
        if teamid not in d:
            d[teamid] = line[team_namex]

    return d

In [104]:
# builds a dictionary -> {year: {team_name: Team, ...}, ...}
def build_years_dict():
    # create the teamid dictionary for refference
    team_lines = readcsv('nfl_teams.csv')
    team_ref = build_teamid_ref(team_lines[0::2])#returns dict of teams
    
    years = {}
    for yr in range(startyr, (endyr + 1)):
        # add year to dict and initialize a dict for teams
        years[yr] = {}
        # gather game data for year
        #yr_file = path + str(yr) + ".csv"
        games = readcsv('nfl_attendance.csv')
        #games = readcsv(yr_file)
        for game in games[0::2]:
            # validate data
            #print(game[0])
            
            
            if game[attx] != 'Bye':
                if game[attx] and int(game[attx]) > 0:
                    # check if team is in team dict or not
                    if game[4] == '@':
                        hteam = game[5]
                if game[attx] and int(game[attx]) > 0:
                    # check if team is in team dict or not
                    if game[4] == '@':
                        hteam = game[5]
                    else:
                        hteam = game[3]

                    teamid = hteam

                    team_name = team_ref[teamid]
                    
                    if team_name not in years[yr]:
                        years[yr][team_name] = Team(team_name, teamid, yr)
                    # add game data to proper team
                    years[yr][team_name].addGame(game)

            for yr in years:
                for team in years[yr]:
                    team = years[yr][team]
                    team.calcStandardDeviation()
                    team.setAvgAtt()
                    
    
        return years


In [105]:
# Analyzes game data to see if high attendance number translate to higher team performance
def supported_percentage(years):
    supported = 0
    games_analyzed = 0
    for y in years:
        year = years[y]
        for t in year:
            team = year[t]
            for g in range(0, len(team.games)):
                game = team.games[g]
                att_dev = team.getAttDeviation(game)
                # check if high/low attendance translates to win/loss
                if abs(att_dev) > team.att_std:
                    games_analyzed += 1
                    if att_dev > 0 and team.win(game):
                        supported += 1
                    elif att_dev < 0 and not team.win(game):
                        supported += 1
    # return the percentage of games that supports: high attendance -> high team performance
    preNormalized = (supported / games_analyzed) * 100
    pctSupported = (preNormalized - 50) * 2
    return round(pctSupported, 2)

In [106]:
years = build_years_dict()

In [107]:
sp = supported_percentage(years)
print("The relation of high/low attendance to win/loss of game is", sp, "% supported.")


The relation of high/low attendance to win/loss of game is 29.78 % supported.
